In [ ]:
using Statistics

function build_bar_by_cumsum(candles, condition, threshold)
    n = size(candles, 1)
    bars = zeros(Float64, n, 6)
    bar_index = 1
    
    # 初始化第一个bar
    bar_timestamp = candles[1, 1]
    bar_open = candles[1, 2]
    bar_close = candles[1, 3]
    bar_high = candles[1, 4]
    bar_low = candles[1, 5]
    bar_volume = candles[1, 6]
    bar_cumsum = condition[1]
    
    for i in 2:n
        if bar_cumsum <= threshold
            bar_cumsum += condition[i]
            bar_timestamp = max(bar_timestamp, candles[i, 1])
            bar_volume += candles[i, 6]
            bar_high = max(bar_high, candles[i, 4])
            bar_low = min(bar_low, candles[i, 5])
            bar_close = candles[i, 3]
        else
            bars[bar_index, 1] = bar_timestamp
            bars[bar_index, 2] = bar_open
            bars[bar_index, 3] = bar_close
            bars[bar_index, 4] = bar_high
            bars[bar_index, 5] = bar_low
            bars[bar_index, 6] = bar_volume
            bar_index += 1
            
            bar_timestamp = candles[i, 1]
            bar_open = candles[i, 2]
            bar_close = candles[i, 3]
            bar_high = candles[i, 4]
            bar_low = candles[i, 5]
            bar_volume = candles[i, 6]
            bar_cumsum = condition[i]
        end
    end
    
    return bars[1:bar_index-1, :]
end

function compute_kurtosis(merged_bars)
    n = size(merged_bars, 1)
    if n <= 5
        return Inf
    end
    
    close_prices = merged_bars[:, 3]
    log_returns = log.(close_prices[6:end] ./ close_prices[1:end-5])
    
    if length(log_returns) == 0
        return Inf
    end
    
    mean_ret = mean(log_returns)
    std_ret = std(log_returns)
    if std_ret == 0
        return Inf
    end
    
    standardized = (log_returns .- mean_ret) ./ std_ret
    n = length(standardized)
    kurt = sum(standardized.^4) / n
    
    return kurt
end

compute_kurtosis (generic function with 1 method)

In [2]:
using NPZ
candles = npzread("data/btc_1m.npy")
candles = candles[candles[:, 6].>0, :]
size(candles)

(2891336, 6)

In [4]:
size(candles, 1) ÷ 360

8031

In [1]:
using Statistics

mean([1,2,3,4])

2.5

In [ ]:
res = build_bar_by_cumsum(candles, condition, 100)

28543×6 Matrix{Float64}:
 1.57784e12    7189.43         7213.42       …    7170.15        5609.39
 1.57785e12    7213.42         7229.93            7200.0         5010.0
 1.57785e12    7230.17         7221.18            7211.22        2664.05
 1.57786e12    7220.99         7225.42            7214.0         1876.03
 1.57787e12    7225.4          7192.17            7181.0         4132.77
 1.57787e12    7191.99         7188.83       …    7174.0         3789.04
 1.57788e12    7189.58         7222.95            7189.58        3263.02
 1.57789e12    7222.95         7218.0             7185.01        4581.05
 1.57789e12    7218.41         7225.0             7204.0         3946.51
 1.5779e12     7225.38         7221.29            7210.81        5670.22
 ⋮                                           ⋱                      ⋮
 1.75076e12  104692.0             1.05182e5          1.04571e5  12162.6
 1.75076e12       1.05182e5       1.05281e5  …  104958.0         6252.38
 1.75077e12       1.05281e5    

In [ ]:
compute_kurtosis(res[1:end÷3, :])

25.216747298944743